<img src="http://csdms.colorado.edu/mediawiki/images/CSDMS_high_res_weblogo.jpg">

# HydroTrend Study with Sampling

[HydroTrend](https://csdms.colorado.edu/wiki/Model:HydroTrend) is a numerical model that creates synthetic river discharge and sediment load time series as a function of climate trends and basin morphology.

In this example, we'll perform a [sampling](https://dakota.sandia.gov//sites/default/files/docs/6.4/html-ref/method-sampling.html) study
to evaluate how varying two HydroTrend input parameters:

* `starting_mean_annual_temperature` (_T_) and
* `total_annual_precipitation` (_P_)

affects the median values of three output parameters

* discharge at the river mouth (_Q_),
* long-term suspended sediment load at the river mouth (_Qs_), and
* daily bedload at the river mouth (_Qb_)

over a 10-year run.

Before we start, make sure that you've installed Dakota, HydroTrend, and this package on your computer, using the instructions in the [README](https://github.com/csdms/dakota/blob/master/README.md) file.

Start by importing the Dakota class.

In [ ]:
from dakotathon import Dakota

Create a Dakota instance to perform a sampling study with HydroTrend using uniform uncertain variables.

In [ ]:
d = Dakota(method='sampling', variables='uniform_uncertain', plugin='hydrotrend')

Define the HydroTrend input variables (_T_ and _P_) to be used in the study.
We assume that we don't know their exact values,
so we set ranges,
approximately &pm;10% of their default values,
and allow them to vary uniformly over this range.

In [ ]:
d.variables.descriptors = ['starting_mean_annual_temperature', 'total_annual_precipitation']  # T and P
d.variables.lower_bounds = [12.8, 1.4]
d.variables.upper_bounds = [15.8, 1.8]

Use [Latin hypercube sampling](https://dakota.sandia.gov/sites/default/files/docs/6.4/html-ref/topic-sampling.html)
to obtain 24 samples from the _T-P_ parameter space.
For testing, set a seed for the random number generator so we get the same samples each time we run the experiment.

In [ ]:
d.method.sample_type = 'lhs'
d.method.samples = 24
d.method.seed = 17

Define the HydroTrend outputs to be used in the study, as well as the statistics to be calculated from them.

In [ ]:
d.responses.response_descriptors = ['Q_median', 'Qs_median', 'Qb_median']
d.responses.response_files = ['HYDROASCII.Q', 'HYDROASCII.QS', 'HYDROASCII.QB']
d.responses.response_statistics = ['median', 'median', 'median']

HydroTrend requires a set of files to run.
They're included in the **data** directory of the repository containing this example.
They can also be obtained directly from the HydroTrend [GitHub repository](https://github.com/mcflugen/hydrotrend/tree/add-bmi-metadata).
Set paths to these files with the following statements.

In [ ]:
import os

data_dir = os.path.join(os.getcwd(), 'data')
template_file = os.path.join(data_dir, 'hydrotrend.in.tmpl')
parameters_file = os.path.join(data_dir, 'parameters.yaml')
hypsometry_file = os.path.join(data_dir, 'HYDRO0.HYPS')

The *template file* provides the configuration file for HydroTrend, but with all parameter values replaced by variables in the form `{parameter_name}`. The *parameters file* provides descriptions, ranges, and default values for all of the parameters represented in the template file. The *hypsometry file* describes the change in elevation along the river's course from source to sea.

From the template and parameters files,
we can create an input file that HydroTrend can run.
Included in the CSDMS Dakota package is a routine that replaces the variables in the template file with default values from the parameters file.
Import this routine and use it to create a HydroTrend input file.

In [ ]:
from dakotathon.plugins.base import write_dflt_file

default_input_file = write_dflt_file(template_file, parameters_file, run_duration=10*365)
print default_input_file

Next, we must replace the default values for the variables for `starting_mean_annual_temperature` and `total_annual_precipitation` with variable names for Dakota to substitute into. The CSDMS Dakota package also includes a routine to do this. Import this routine and use it to create a Dakota template file.

In [ ]:
from dakotathon.plugins.base import write_dtmpl_file

dakota_template_file = write_dtmpl_file(template_file, default_input_file, d.variables.descriptors)
print dakota_template_file

Associate the Dakota template file and the HydroTrend hypsometry file with the Dakota instance.

In [ ]:
d.template_file = dakota_template_file
d.auxiliary_files = hypsometry_file

Call the setup method to create the Dakota input file, **dakota.in**.

In [ ]:
d.setup()

Run the experiment.

In [ ]:
d.run()

Check the output. First, the **dakota.dat** file.

In [ ]:
%cat dakota.dat

Next, the **dakota.out** file.
At the end of this file,
statistics calculated by Dakota are reported.

In [ ]:
%cat dakota.out

Note that _Qs_ is highly correlated with _T_, while _Q_ and _Qb_ are highly correlated with _P_.